<a href="https://colab.research.google.com/github/m-mazurkiewicz/PWR_MachineLearning/blob/AlexNet_master/AlexNetArchitecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
  from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D as Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

import sys
from keras.preprocessing.image import ImageDataGenerator, array_to_img, save_img
from matplotlib import pyplot as plt
import skimage.io as io
import numpy as np
import os
from tqdm import tqdm
from google.colab import drive

Using TensorFlow backend.


Wait for 8 seconds...
TensorBoard link:
https://ce8aeb27.ngrok.io


In [0]:
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [0]:
num_of_classes = 4
base_dir_processed = '/content/gdrive/My Drive/PWr_AlexNet_data/processed/'
data_set = 'no-padding/resize/'

In [0]:
BATCH_SIZE = 128
VALIDATION_SPLIT = 0.1

In [0]:
image_gen = ImageDataGenerator(
    featurewise_center=True,
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=(0.5,1.5),
    shear_range=0.01,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=VALIDATION_SPLIT
)

In [0]:
number_of_images_for_fit = -1

all_images = []
for class_name in os.listdir(base_dir_processed + data_set + 'train'):
  for image_path in tqdm(os.listdir(base_dir_processed + data_set + 'train/' + class_name)[:number_of_images_for_fit]):
    img = io.imread(base_dir_processed + data_set + 'train/' + class_name + '/' + image_path)
    all_images.append(img)
x_train = np.array(all_images)

image_gen.fit(x_train)

generator = image_gen.flow_from_directory(
    base_dir_processed + data_set + '/' + 'train',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

validation_generator = image_gen.flow_from_directory(
    base_dir_processed + data_set + '/' + 'train',
        target_size=(227,227),
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')

100%|██████████| 1109/1109 [00:04<00:00, 252.42it/s]


In [0]:
# image_gen_val = ImageDataGenerator(featurewise_center=True)
# image_gen_val.fit(x_train)
# val_generator = image_gen_val.flow_from_directory(
#     base_dir_processed + data_set + '/' + 'test',
#         target_size=(227,227),
#         batch_size=128,
#         class_mode='categorical')

Found 1304 images belonging to 4 classes.


#Model

In [0]:
model = Sequential()

In [0]:
model.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

In [0]:
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

In [0]:
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

In [0]:
model.add(Flatten())

In [0]:
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))

In [0]:
model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

In [0]:
model.add(Dense(4))
model.add(Activation('softmax'))

In [0]:
model.summary()

In [0]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [0]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=2, min_lr=0.001)
hist = model.fit_generator(generator, steps_per_epoch=(5213*(1-VALIDATION_SPLIT)) // BATCH_SIZE, validation_data=validation_generator, validation_steps=4, epochs=100, verbose=1,
                           callbacks=[reduce_lr])

#Model saving

In [0]:
base_dir_saving = '/content/gdrive/My Drive/PWr_AlexNet_models/'
model_path = data_set + '{}/'.format(BATCH_SIZE)
os.makedirs(base_dir_saving+model_path,  exist_ok=True)

In [0]:
model.save(base_dir_saving+model_path+'base_alexnet.h5')

# Plotting

In [0]:
plt.figure(figsize=(8, 8), dpi=320)

plt.subplot(211)
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplots_adjust(hspace = .3)

# Plot training & validation loss values
plt.subplot(212)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.savefig(base_dir_saving+model_path+'base_alexnet.pdf')
plt.show()